In [ ]:
from datetime import datetime, timedelta,date
import pandas as pd
import numpy as np
import jenkspy
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
pyoff.init_notebook_mode()

In [ ]:
tx_data  = pd.read_csv('C:/Users/GK/Part4.csv')
tx_j = pd.read_csv('jijoan.csv')
tx_j = tx_j.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)
menu = tx_j.groupby(['帳戶名稱','cust_rank']).nunique().reset_index(level = ['帳戶名稱','cust_rank'])
tx_data = pd.merge(tx_data,menu[['帳戶名稱','cust_rank']], on='帳戶名稱')
olo = ['產品代碼','二元類型','auto_index','acc_money','avg_money','self_index','comb','diff','機會名稱']
tx_data = tx_data.drop(olo,axis=1)
tx_data['comb'] = tx_data['帳戶名稱'] +tx_data['產品名稱']
tx_data = tx_data.sort_values(by= ['comb','結帳日期']).reset_index(drop=True)
tx_data.loc[tx_data['類型']!='新客戶','類型'] = '舊客戶'
tx_data['結帳日期']=pd.to_datetime(tx_data['結帳日期'])
tx_data['year'] = tx_data['結帳日期'].dt.year
tx_data['month'] = tx_data['結帳日期'].dt.month 
iangDpienC_list = ['樣品切削油類','樣品切削油類、潤滑油類','樣品液壓油類','樣品滑道油類','樣品潤滑油類','樣品防銹油類','未分類樣品']
for i in iangDpienC_list:
    tx_data.loc[tx_data['產品描述']==i,'產品描述'] = '樣品'

ciiBta_list = ['其他','包裝材料類','回收類','服務類','未分類','配件類']
for i in ciiBta_list:
    tx_data.loc[tx_data['產品描述']==i,'產品描述'] = '其他'

In [ ]:
gg= tx_data.groupby('comb').nunique()
gg[gg['總量']!=1]

In [ ]:
tx_train = tx_data[(tx_data.結帳日期 < pd.to_datetime('2019-6-1')) & (tx_data.結帳日期 >= pd.to_datetime('2017-1-1'))].reset_index(drop=True)
tx_test = tx_data[(tx_data.結帳日期 >= pd.to_datetime('2019-6-1')) & (tx_data.結帳日期 < pd.to_datetime('2020-9-1'))].reset_index(drop=True)
x = set(pd.DataFrame(tx_train['帳戶名稱'].unique())[0])
y = set(pd.DataFrame(tx_test['帳戶名稱'].unique())[0])
print("len of train:", len(x), " len of test:", len(y))
print("intersection: ",len(x.intersection(y))," difference: ", len(y.difference(x)))

In [ ]:
#REC (答案)
tx_data['結帳日期'] = pd.to_datetime(tx_data['結帳日期'])
tx_data = tx_data.sort_values(by= ['comb','結帳日期']).reset_index(drop=True)
freq_list=[]
for i in range(tx_data.shape[0]):
    if i == tx_data.shape[0]-1:
        freq = np.nan
    else:
        if tx_data.loc[i,'comb'] != tx_data.loc[i+1,'comb']:
            freq = np.nan
        else:
            interval=tx_data.loc[i+1,'結帳日期']-tx_data.loc[i,'結帳日期']
            freq = interval.days 
    freq_list.append(freq)
tx_data['NextPurchaseDay']=freq_list
pjc_data = tx_data.copy()
opo = ['數量','規格','總量']
for i in opo:
    del pjc_data[i]
pjc_data = pjc_data.groupby(['帳戶名稱','結帳日期']).sum().reset_index(level=['帳戶名稱','結帳日期'])
pjc_data['結帳日期']=pd.to_datetime(pjc_data['結帳日期'])
pjc_data['year'] = pjc_data['結帳日期'].dt.year
pjc_data['month'] = pjc_data['結帳日期'].dt.month 

In [ ]:
#REC(答案)---------------------------------------------------------------------------
pjc_data['結帳日期'] = pd.to_datetime(pjc_data['結帳日期'])
pjc_data = pjc_data.sort_values(by= ['帳戶名稱','結帳日期']).reset_index(drop=True)
freq_list=[]
for i in range(pjc_data.shape[0]):
    if i == pjc_data.shape[0]-1:
        freq = np.nan
    else:
        if pjc_data.loc[i,'帳戶名稱'] != pjc_data.loc[i+1,'帳戶名稱']:
            freq = np.nan
        else:
            interval=pjc_data.loc[i+1,'結帳日期']-pjc_data.loc[i,'結帳日期']
            freq = interval.days 
    freq_list.append(freq)
pjc_data['NextPurchaseDay']=freq_list

#FREQUENCY---------------------------------------------------------------------------
freq_list=[]
record = 0
for i in range(pjc_data.shape[0]):
    if i == 0:
        freq = 1
        record = 1
    else:
        if pjc_data.loc[i,'帳戶名稱'] != pjc_data.loc[i-1,'帳戶名稱']:
            freq = 1
            record = 1
        else:
            record = record + 1
            freq = int(record)
    freq_list.append(freq)
pjc_data['freq']=freq_list

In [ ]:
max_ = pd.DataFrame(pjc_data.groupby('帳戶名稱').max()['NextPurchaseDay'])
avg_ = pd.DataFrame(pjc_data.groupby('帳戶名稱').mean()['NextPurchaseDay'])
median_ = pd.DataFrame(pjc_data.groupby('帳戶名稱').median()['NextPurchaseDay'])
Recency_ = pd.DataFrame(pjc_data.drop_duplicates(subset=['帳戶名稱'],keep = 'last'))
Money_ = pd.DataFrame(pjc_data.groupby('帳戶名稱').sum()['總價'])
menu = max_.merge(avg_, on='帳戶名稱', how='left').merge(median_, on='帳戶名稱', how='left').merge(Money_, on='帳戶名稱', how='left').merge(Recency_[['結帳日期','帳戶名稱']], on='帳戶名稱', how='left')
menu.columns = ['帳戶名稱','max','mean','med','Money','Recency']
pjc_data = pd.merge(pjc_data,menu, on='帳戶名稱')
pjc_data['Recency'] = pjc_data['Recency'].max()-pjc_data['Recency']

In [ ]:
pjc_train = pjc_data[(pjc_data.結帳日期 < pd.to_datetime('2019-6-1')) & (pjc_data.結帳日期 >= pd.to_datetime('2017-1-1'))].reset_index(drop=True)
pjc_test = pjc_data[(pjc_data.結帳日期 >= pd.to_datetime('2019-6-1')) & (pjc_data.結帳日期 < pd.to_datetime('2020-9-1'))].reset_index(drop=True)
x = set(pd.DataFrame(pjc_train['帳戶名稱'].unique())[0])
y = set(pd.DataFrame(pjc_test['帳戶名稱'].unique())[0])
print("len of train:", len(x), " len of test:", len(y))
print("intersection: ",len(x.intersection(y))," difference: ", len(y.difference(x)))

In [ ]:
pjc_data['Recency'] = pjc_data['Recency'].dt.days
pjc_data.loc[pjc_data['NextPurchaseDay'].isnull(),'NextPurchaseDay'] = pjc_data['Recency']
pjc_data['max'] = pjc_data['max'].fillna(1340)
pjc_data['mean'] = pjc_data['mean'].fillna(1340)
pjc_data['med'] = pjc_data['med'].fillna(1340)

In [ ]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final
def goodness_of_variance_fit(array, classes):
    # get the break points
    classes = jenkspy.jenks_breaks(array, classes)

    # do the actual classification
    classified = np.array([classify(i, classes) for i in array])

    # max value of zones
    maxz = max(classified)

    # nested list of zone indices
    zone_indices = [[idx for idx, val in enumerate(classified) if zone + 1 == val] for zone in range(maxz)]

    # sum of squared deviations from array mean
    sdam = np.sum((array - array.mean()) ** 2)

    # sorted polygon stats
    array_sort = [np.array([array[index] for index in zone]) for zone in zone_indices]

    # sum of squared deviations of class means
    sdcm = sum([np.sum((classified - classified.mean()) ** 2) for classified in array_sort])

    # goodness of variance fit
    gvf = (sdam - sdcm) / sdam

    return gvf

def classify(value, breaks):
    for i in range(1, len(breaks)):
        if value < breaks[i]:
            return i
    return len(breaks) - 1

In [ ]:
array_org = np.array(pjc_data[['Recency']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .99:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF)+1)
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.95]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()
bins = jenkspy.jenks_breaks(array_org, 6)
labels = [x for x in range(len(bins)-1)]
pjc_data['RecencyCluster'] = pd.cut(pjc_data['Recency'], bins=bins, labels=labels)
pjc_data.groupby('RecencyCluster')['Recency'].describe()
pjc_data = order_cluster('RecencyCluster', 'Recency',pjc_data,False)

In [ ]:
array_org = np.array(pjc_data[['freq']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .99 and nclasses<9:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF)+1)
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.95]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()
bins = jenkspy.jenks_breaks(array_org, 6)
labels = [x for x in range(len(bins)-1)]
pjc_data['FrequencyCluster'] = pd.cut(pjc_data['freq'], bins=bins, labels=labels)
pjc_data.groupby('FrequencyCluster')['freq'].describe()
pjc_data = order_cluster('FrequencyCluster', 'freq',pjc_data,True)

In [ ]:
array_org = np.array(pjc_data[['Money']].T)[0]
list_GVF = [] 
gvf = 0.0
nclasses = 2
while gvf < .99 and nclasses<9:
    gvf = goodness_of_variance_fit(array_org, nclasses)
    print(nclasses,gvf)
    list_GVF = list_GVF + [gvf]
    nclasses += 1
plt.figure()
plt.xlim(2,len(list_GVF)+1)
plt.plot([x+2 for x in range(len(list_GVF))], list_GVF)
plt.plot([x+2 for x in range(len(list_GVF))], [0.95]*len(list_GVF))
plt.xlabel("Number of cluster")
plt.show()
bins = jenkspy.jenks_breaks(array_org, 6)
labels = [x for x in range(len(bins)-1)]
pjc_data['RevenueCluster'] = pd.cut(pjc_data['Money'], bins=bins, labels=labels)
pjc_data.groupby('RevenueCluster')['Money'].describe()
pjc_data = order_cluster('RevenueCluster', 'Money',pjc_data,True)

In [ ]:
pjc_data.groupby('FrequencyCluster')['freq'].describe()

In [ ]:
pjc_data.groupby('RecencyCluster')['freq'].describe()

In [ ]:
pjc_data.groupby('RevenueCluster')['freq'].describe()

In [ ]:
pjc_data['OverallScore'] = pjc_data['RecencyCluster'] + pjc_data['FrequencyCluster'] + pjc_data['RevenueCluster']

In [ ]:
pjc_data.groupby('OverallScore')['Recency','freq','Money'].mean()

In [ ]:
pjc_data['Segment'] = 'Low-Value'
pjc_data.loc[pjc_data['OverallScore']>3,'Segment'] = 'Mid-Value' 
pjc_data.loc[pjc_data['OverallScore']>7,'Segment'] = 'High-Value' 

In [ ]:
pjc_data[pjc_data['客戶自主下單']>1]=1
del pjc_data['循環天數']

In [ ]:
tx_j = pd.read_csv('jijoan.csv')
tx_j = tx_j.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)
menu = tx_j.groupby(['帳戶名稱','cust_rank','分級']).nunique().reset_index(level = ['帳戶名稱','cust_rank','分級'])
pjc_data = pd.merge(pjc_data,menu[['帳戶名稱','cust_rank','分級']], on='帳戶名稱')

In [ ]:
pjc_data.nunique()

In [ ]:
pjc_data.info()

In [ ]:
pjc_data.to_csv('pjc_data.csv',index_label=False)